In [1]:
import os

In [2]:
cd ../

/home/asus/Documents/Chest-Cancer-Classification/Project


In [3]:
%pwd

'/home/asus/Documents/Chest-Cancer-Classification/Project'

## Update Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int



## Update Configuration

In [5]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

## Update components

In [7]:
!pip install tensorflow

     |████████████████████████████████| 479.6 MB 37 kB/s  eta 0:00:01     |███████████████▊                | 235.5 MB 3.9 MB/s eta 0:01:03    |██████████████████████████████  | 448.2 MB 1.0 MB/s eta 0:00:31     |██████████████████████████████▋ | 459.2 MB 9.4 MB/s eta 0:00:032 MB 982 kB/s eta 0:00:17
     |████████████████████████████████| 17.3 MB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 4.2 MB/s eta 0:00:011
     |████████████████████████████████| 83 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 24.5 MB 152 kB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 4.1 MB/s eta 0:00:011
     |████████████████████████████████| 2.4 MB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 5.3 MB/s eta 0:00:01
     |████████████

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

2024-08-28 14:42:29.317860: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-28 14:42:29.379313: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-28 14:42:29.380073: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-28 14:42:30.333057: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:


class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
    def get_base_model(self):
        self.model = tf.keras.applications.vgg19.VGG19(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)


    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
    


    



## Update the pipeline

In [10]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e


[2024-08-28 14:42:58,484: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-28 14:42:58,490: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-28 14:42:58,492: INFO: common: created directory at: artifacts]
[2024-08-28 14:42:58,494: INFO: common: created directory at: artifacts/prepare_base_model]
80134624/80134624 [==============================] - 22s 0us/step
[2024-08-28 14:43:21,013: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


/home/asus/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     